In [66]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import geopandas as gpd

In [2]:
df = pd.read_csv('Water_Point_Data_Exchange__WPDx-Basic_.csv', dtype='object')

In [3]:
df.shape

(89717, 52)

In [4]:
def distance_between_two_points(lat1, lat2, lon1, lon2):

     """
     A procedure to get the distance between two points with latitdues and longitudes
     INPUTS:
        lat1: -> latitude of the first point in degree 
        lat2: -> latitude of the second point in degree 
        lon1: -> longitude of the first point in degree 
        lon2: -> longitude of the second point in degree 
     """    
    # The numpy module contains a function named
    # radians which converts from degrees to radians.

     #lon1a = np.radians(lon1)
    # lon2a = np.radians(lon2)
     lat1a = np.radians(lat1)
     lat2a = np.radians(lat2)
      
    # Haversine formula
    
     dlon = np.radians(lon2 - lon1)
     dlat = np.radians(lat2 - lat1)
     a = np.sin(dlat / 2)**2 + np.cos(lat1a) * np.cos(lat2a) * np.sin(dlon / 2)**2
 
    
     c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
     r = 6371
      
     # calculate the result

     return np.round(c * r, 6)

In [5]:
df.head()

,row_id,#source,#lat_deg,#lon_deg,#report_date,#status_id,#water_source,#water_source_clean,#water_tech_clean,#water_tech,...,clean_adm2,clean_adm3,water_tech_original,water_source_original,lat_deg_original,lon_deg_original,management_clean,management_original,water_source_category,water_tech_category
0,168928,"Federal Ministry of Water Resources, Nigeria",6.2197817,7.006075,05/27/2015 12:00:00 AM,Yes,Improved Tube well or borehole,Borehole,Mechanized Pump,Motorised,...,AwkaSout,NaN,motorised,improved tube well or borehole,NaN,NaN,Other,other,Well,Mechanized Pump
1,169214,"Federal Ministry of Water Resources, Nigeria",6.2172267,6.90776,05/22/2015 12:00:00 AM,No,Improved Tube well or borehole,Borehole,Mechanized Pump,Motorised,...,Oyi,NaN,motorised,improved tube well or borehole,NaN,NaN,NaN,NaN,Well,Mechanized Pump
2,169259,"Federal Ministry of Water Resources, Nigeria",5.8981991,6.8390944,05/03/2015 12:00:00 AM,Yes,Improved Tube well or borehole,Borehole,Hand Pump,Hand pump,...,NnewiSou,NaN,hand pump,improved tube well or borehole,NaN,NaN,Health Care Facility,institutional management - health care facility,Well,Hand Pump
3,207839,"Federal Ministry of Water Resources, Nigeria",7.8310683,8.8338717,08/27/2015 12:00:00 AM,No,Improved Tube well or borehole,Borehole,Hand Pump,Manual,...,Guma,NaN,manual,improved tube well or borehole,NaN,NaN,NaN,NaN,Well,Hand Pump
4,208157,"Federal Ministry of Water Resources, Nigeria",7.0277082,8.5691409,08/23/2015 12:00:00 AM,Yes,Improved Tube well or borehole,Borehole,Hand Pump,Hand pump,...,Konshish,NaN,hand pump,improved tube well or borehole,NaN,NaN,Other,other,Well,Hand Pump


In [8]:
# Drop columns  with all NaN, single values, irrelevant columns
df.dropna(how='all', axis=1, inplace=True)
cols_to_drop = ['row_id', '#source', '#facility_type', '#report_date', '#country_name',  '#data_lnk', '#converted', '#country_id', '#install_year', 'created_timestamp', 'updated', 'public_data_source', 'water_tech_category', 'water_tech_original', 'count', 'management_original', 'public_data_source', 'clean_country_id', 'clean_country_name']
df.drop(cols_to_drop, axis =1, inplace=True)
df.shape


(89717, 29)

In [43]:
# Drop more columns with duplication, majority of NaN
cols_to_drop1 = ['#adm3', '#water_source', '#water_tech', '#installer', '#management', '#fecal_coliform_value', '#activity_id', '#scheme_id', '#orig_lnk', 'clean_adm3', 'water_source_original']
df.drop(cols_to_drop1, axis =1, inplace=True)
df.shape


(89717, 18)

In [44]:
df.columns

Index(['#lat_deg', '#lon_deg', '#status_id', '#water_source_clean',
       '#water_tech_clean', '#adm1', '#adm2', '#pay', '#status',
       '#subjective_quality', '#notes', '#photo_lnk', 'lat_lon_deg',
       'New Georeferenced Column', 'clean_adm1', 'clean_adm2',
       'management_clean', 'water_source_category'],
      dtype='object')

In [ ]:
# Drop more 

In [46]:
df['clean_adm1'].head()
#df['#water_tech'].value_counts(dropna=False)

0    Anambra
1    Anambra
2    Anambra
3      Benue
4      Benue
Name: clean_adm1, dtype: object

In [36]:
df['water_source_original'].isnull().sum().sum()

7

In [49]:
# Rename columns with # prefixes
mapper = {'#lat_deg': 'latitude', '#lon_deg': 'longitude', '#status_id' :'status_id', '#water_source_clean': 'water_source', '#water_tech_clean': 'water_technology', '#adm1': 'state',
       '#adm2': 'lga', '#pay': 'pay', '#status': 'status', '#subjective_quality': 'subjective_quality', '#notes': 'location', '#photo_lnk': 'photo_lnk','clean_adm1': 'state' , 'clean_adm2':'lga'}
df= df.rename(columns=mapper)

In [50]:
df.head()

,latitude,longitude,status_id,water_source,water_technology,state,lga,pay,status,subjective_quality,location,photo_lnk,lat_lon_deg,New Georeferenced Column,state,lga,management_clean,water_source_category
0,6.2197817,7.006075,Yes,Borehole,Mechanized Pump,Anambra,Njikoka,No,Functional (and in use),Acceptable quality,Etiti 1,https://akvoflow-55.s3.amazonaws.com/images/8b...,"(6.2197817°, 7.006075°)",POINT (7.006075 6.2197817),Anambra,AwkaSout,Other,Well
1,6.2172267,6.90776,No,Borehole,Mechanized Pump,Anambra,Oyi,No,Non-functional Technical breakdown,Acceptable quality,Ajakpani Umunya,https://akvoflow-55.s3.amazonaws.com/images/37...,"(6.2172267°, 6.90776°)",POINT (6.90776 6.2172267),Anambra,Oyi,NaN,Well
2,5.8981991,6.8390944,Yes,Borehole,Hand Pump,Anambra,Ihala,No,Functional (and in use),Acceptable quality,Ubahumonum Okija,NaN,"(5.8981991°, 6.8390944°)",POINT (6.8390944 5.8981991),Anambra,NnewiSou,Health Care Facility,Well
3,7.8310683,8.8338717,No,Borehole,Hand Pump,Benue,Guma,No,Non-functional Technical breakdown,No because of Taste,Iordye,NaN,"(7.8310683°, 8.8338717°)",POINT (8.8338717 7.8310683),Benue,Guma,NaN,Well
4,7.0277082,8.5691409,Yes,Borehole,Hand Pump,Benue,Konshisha,No,Functional (and in use),Acceptable quality,Manta,NaN,"(7.0277082°, 8.5691409°)",POINT (8.5691409 7.0277082),Benue,Konshish,Other,Well


In [55]:
df['latitude'] = df['latitude'].astype('float')
df['longitude'] = df['longitude'].astype('float')

In [56]:
# Create  hypothetical resident's latitude and longitude
np.random.seed(0) 
#df.insert(0, 'resident_latitude', df.latitude + 0.0051*np.random.rand(df.shape[0]))
df.insert(1, 'resident_longitude', df.longitude + 0.00982*np.random.rand(df.shape[0]))


In [57]:
df.head()

,resident_latitude,resident_longitude,latitude,longitude,status_id,water_source,water_technology,state,lga,pay,...,subjective_quality,location,photo_lnk,lat_lon_deg,New Georeferenced Column,state,lga,management_clean,water_source_category,longiude
0,6.222581,7.011464,6.219782,7.006075,Yes,Borehole,Mechanized Pump,Anambra,Njikoka,No,...,Acceptable quality,Etiti 1,https://akvoflow-55.s3.amazonaws.com/images/8b...,"(6.2197817°, 7.006075°)",POINT (7.006075 6.2197817),Anambra,AwkaSout,Other,Well,7.006075
1,6.220874,6.914783,6.217227,6.907760,No,Borehole,Mechanized Pump,Anambra,Oyi,No,...,Acceptable quality,Ajakpani Umunya,https://akvoflow-55.s3.amazonaws.com/images/37...,"(6.2172267°, 6.90776°)",POINT (6.90776 6.2172267),Anambra,Oyi,NaN,Well,6.907760
2,5.901273,6.845014,5.898199,6.839094,Yes,Borehole,Hand Pump,Anambra,Ihala,No,...,Acceptable quality,Ubahumonum Okija,NaN,"(5.8981991°, 6.8390944°)",POINT (6.8390944 5.8981991),Anambra,NnewiSou,Health Care Facility,Well,6.839094
3,7.833847,8.839222,7.831068,8.833872,No,Borehole,Hand Pump,Benue,Guma,No,...,No because of Taste,Iordye,NaN,"(7.8310683°, 8.8338717°)",POINT (8.8338717 7.8310683),Benue,Guma,NaN,Well,8.833872
4,7.029869,8.573301,7.027708,8.569141,Yes,Borehole,Hand Pump,Benue,Konshisha,No,...,Acceptable quality,Manta,NaN,"(7.0277082°, 8.5691409°)",POINT (8.5691409 7.0277082),Benue,Konshish,Other,Well,8.569141


In [60]:
# Create an id column
df.drop('id', axis=1, inplace=True)
df.insert(0, 'id', list(range(1, 1+df.shape[0])))

In [61]:
df.head()

,id,resident_latitude,resident_longitude,latitude,longitude,status_id,water_source,water_technology,state,lga,...,subjective_quality,location,photo_lnk,lat_lon_deg,New Georeferenced Column,state,lga,management_clean,water_source_category,longiude
0,1,6.222581,7.011464,6.219782,7.006075,Yes,Borehole,Mechanized Pump,Anambra,Njikoka,...,Acceptable quality,Etiti 1,https://akvoflow-55.s3.amazonaws.com/images/8b...,"(6.2197817°, 7.006075°)",POINT (7.006075 6.2197817),Anambra,AwkaSout,Other,Well,7.006075
1,2,6.220874,6.914783,6.217227,6.907760,No,Borehole,Mechanized Pump,Anambra,Oyi,...,Acceptable quality,Ajakpani Umunya,https://akvoflow-55.s3.amazonaws.com/images/37...,"(6.2172267°, 6.90776°)",POINT (6.90776 6.2172267),Anambra,Oyi,NaN,Well,6.907760
2,3,5.901273,6.845014,5.898199,6.839094,Yes,Borehole,Hand Pump,Anambra,Ihala,...,Acceptable quality,Ubahumonum Okija,NaN,"(5.8981991°, 6.8390944°)",POINT (6.8390944 5.8981991),Anambra,NnewiSou,Health Care Facility,Well,6.839094
3,4,7.833847,8.839222,7.831068,8.833872,No,Borehole,Hand Pump,Benue,Guma,...,No because of Taste,Iordye,NaN,"(7.8310683°, 8.8338717°)",POINT (8.8338717 7.8310683),Benue,Guma,NaN,Well,8.833872
4,5,7.029869,8.573301,7.027708,8.569141,Yes,Borehole,Hand Pump,Benue,Konshisha,...,Acceptable quality,Manta,NaN,"(7.0277082°, 8.5691409°)",POINT (8.5691409 7.0277082),Benue,Konshish,Other,Well,8.569141


In [62]:
df.drop('longiude', axis=1, inplace=True)

In [63]:
df.head()

,id,resident_latitude,resident_longitude,latitude,longitude,status_id,water_source,water_technology,state,lga,...,status,subjective_quality,location,photo_lnk,lat_lon_deg,New Georeferenced Column,state,lga,management_clean,water_source_category
0,1,6.222581,7.011464,6.219782,7.006075,Yes,Borehole,Mechanized Pump,Anambra,Njikoka,...,Functional (and in use),Acceptable quality,Etiti 1,https://akvoflow-55.s3.amazonaws.com/images/8b...,"(6.2197817°, 7.006075°)",POINT (7.006075 6.2197817),Anambra,AwkaSout,Other,Well
1,2,6.220874,6.914783,6.217227,6.907760,No,Borehole,Mechanized Pump,Anambra,Oyi,...,Non-functional Technical breakdown,Acceptable quality,Ajakpani Umunya,https://akvoflow-55.s3.amazonaws.com/images/37...,"(6.2172267°, 6.90776°)",POINT (6.90776 6.2172267),Anambra,Oyi,NaN,Well
2,3,5.901273,6.845014,5.898199,6.839094,Yes,Borehole,Hand Pump,Anambra,Ihala,...,Functional (and in use),Acceptable quality,Ubahumonum Okija,NaN,"(5.8981991°, 6.8390944°)",POINT (6.8390944 5.8981991),Anambra,NnewiSou,Health Care Facility,Well
3,4,7.833847,8.839222,7.831068,8.833872,No,Borehole,Hand Pump,Benue,Guma,...,Non-functional Technical breakdown,No because of Taste,Iordye,NaN,"(7.8310683°, 8.8338717°)",POINT (8.8338717 7.8310683),Benue,Guma,NaN,Well
4,5,7.029869,8.573301,7.027708,8.569141,Yes,Borehole,Hand Pump,Benue,Konshisha,...,Functional (and in use),Acceptable quality,Manta,NaN,"(7.0277082°, 8.5691409°)",POINT (8.5691409 7.0277082),Benue,Konshish,Other,Well


In [65]:
# Create a column for distance between the resident and nearest water point
distance_in_km = []
for row in df.itertuples(index=False):
    distance_in_km.append(
       distance_between_two_points(row.resident_latitude, row.latitude, row.resident_longitude, row.longitude)
       )

df['distance_in_km'] = distance_in_km

In [76]:
# fill missing values 
df= df.fillna(method='bfill')
df= df.fillna(method='ffill')
df.isnull().sum().sum()

0

In [77]:
df.shape

(89717, 23)

In [72]:
# add hypothetical population column to the dataframe 
np.random.seed(0)
df.insert(22, 'population', np.random.randint(1500, 2830, size= df.shape[0]))

In [78]:
# create a sqlite database engine using sqlalchemy
database_filepath = 'water_points.db'
engine = create_engine(f'sqlite:///{database_filepath}')
# save the df in a table if exists replace
df.to_sql('etlTable', engine, index=False, if_exists='replace')


In [70]:
df.head()

,id,resident_latitude,resident_longitude,latitude,longitude,status_id,water_source,water_technology,state,lga,...,location,photo_lnk,lat_lon_deg,New Georeferenced Column,state,lga,management_clean,water_source_category,distance_in_km,population
0,1,6.222581,7.011464,6.219782,7.006075,Yes,Borehole,Mechanized Pump,Anambra,Njikoka,...,Etiti 1,https://akvoflow-55.s3.amazonaws.com/images/8b...,"(6.2197817°, 7.006075°)",POINT (7.006075 6.2197817),Anambra,AwkaSout,Other,Well,0.672137,2141
1,2,6.220874,6.914783,6.217227,6.907760,No,Borehole,Mechanized Pump,Anambra,Oyi,...,Ajakpani Umunya,https://akvoflow-55.s3.amazonaws.com/images/37...,"(6.2172267°, 6.90776°)",POINT (6.90776 6.2172267),Anambra,Oyi,NaN,Well,0.875902,2195
2,3,5.901273,6.845014,5.898199,6.839094,Yes,Borehole,Hand Pump,Anambra,Ihala,...,Ubahumonum Okija,NaN,"(5.8981991°, 6.8390944°)",POINT (6.8390944 5.8981991),Anambra,NnewiSou,Health Care Facility,Well,0.738556,2330
3,4,7.833847,8.839222,7.831068,8.833872,No,Borehole,Hand Pump,Benue,Guma,...,Iordye,NaN,"(7.8310683°, 8.8338717°)",POINT (8.8338717 7.8310683),Benue,Guma,NaN,Well,0.665510,2382
4,5,7.029869,8.573301,7.027708,8.569141,Yes,Borehole,Hand Pump,Benue,Konshisha,...,Manta,NaN,"(7.0277082°, 8.5691409°)",POINT (8.5691409 7.0277082),Benue,Konshish,Other,Well,0.518188,2401
